In [72]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, IntegerType, ShortType, CharType
from pyspark.sql.functions import col, desc, max_by, min_by

spark = SparkSession.builder.master("local").appName("Babies Notebook").getOrCreate()

In [11]:
# LOAD DATA SET TO SPARK

df = spark.read\
    .option("header", True)\
    .csv('./Baby_Names__Beginning_2007.csv')

DataFrame[Year: string, First Name: string, County: string, Sex: string, Count: string]

In [7]:
# SHOW ORIGINAL SCHEMA AND DATA TYPES

df.printSchema()
df.show()

root
 |-- Year: integer (nullable = true)
 |-- First Name: string (nullable = true)
 |-- County: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Count: integer (nullable = true)

+----+----------+------+---+-----+
|Year|First Name|County|Sex|Count|
+----+----------+------+---+-----+
|2021|    OLIVIA|Albany|  F|   16|
|2021|    AMELIA|Albany|  F|   16|
|2021|    SOPHIA|Albany|  F|   13|
|2021|       AVA|Albany|  F|   13|
|2021|      EMMA|Albany|  F|   12|
|2021|  PENELOPE|Albany|  F|   11|
|2021|   MADISON|Albany|  F|   10|
|2021|   ABIGAIL|Albany|  F|   10|
|2021|    HARPER|Albany|  F|   10|
|2021|      ISLA|Albany|  F|    9|
|2021| CHARLOTTE|Albany|  F|    8|
|2021|  ISABELLA|Albany|  F|    8|
|2021|    GIANNA|Albany|  F|    8|
|2021|   ELEANOR|Albany|  F|    8|
|2021|   LILLIAN|Albany|  F|    8|
|2021|  MADELINE|Albany|  F|    8|
|2021|    EVELYN|Albany|  F|    7|
|2021|     RILEY|Albany|  F|    7|
|2021|    HAILEY|Albany|  F|    7|
|2021|    EVERLY|Albany|  F|    7|

In [24]:
# NORMALIZED COLUMNS NAMES

normalized_columns_names = [col.lower().replace(' ', '_') for col in df.columns]

df = df.toDF(*normalized_columns_names)
df.show()

+----+----------+------+---+-----+
|year|first_name|county|sex|count|
+----+----------+------+---+-----+
|2021|    OLIVIA|Albany|  F|   16|
|2021|    AMELIA|Albany|  F|   16|
|2021|    SOPHIA|Albany|  F|   13|
|2021|       AVA|Albany|  F|   13|
|2021|      EMMA|Albany|  F|   12|
|2021|  PENELOPE|Albany|  F|   11|
|2021|   MADISON|Albany|  F|   10|
|2021|   ABIGAIL|Albany|  F|   10|
|2021|    HARPER|Albany|  F|   10|
|2021|      ISLA|Albany|  F|    9|
|2021| CHARLOTTE|Albany|  F|    8|
|2021|  ISABELLA|Albany|  F|    8|
|2021|    GIANNA|Albany|  F|    8|
|2021|   ELEANOR|Albany|  F|    8|
|2021|   LILLIAN|Albany|  F|    8|
|2021|  MADELINE|Albany|  F|    8|
|2021|    EVELYN|Albany|  F|    7|
|2021|     RILEY|Albany|  F|    7|
|2021|    HAILEY|Albany|  F|    7|
|2021|    EVERLY|Albany|  F|    7|
+----+----------+------+---+-----+
only showing top 20 rows



In [26]:
# CAST CORRECT COLUMN TYPES

schema_map = {
    'year': ShortType(),
    'first_name': StringType(),
    'county': StringType(),
    'sex': CharType(1),
    'count': IntegerType()
}

df_updated_schema = df.select([col(column_schema[0]).cast(schema_map.get(column_schema[0], column_schema[1])) for column_schema in df.dtypes])
df_updated_schema.show()

+----+----------+------+---+-----+
|year|first_name|county|sex|count|
+----+----------+------+---+-----+
|2021|    OLIVIA|Albany|  F|   16|
|2021|    AMELIA|Albany|  F|   16|
|2021|    SOPHIA|Albany|  F|   13|
|2021|       AVA|Albany|  F|   13|
|2021|      EMMA|Albany|  F|   12|
|2021|  PENELOPE|Albany|  F|   11|
|2021|   MADISON|Albany|  F|   10|
|2021|   ABIGAIL|Albany|  F|   10|
|2021|    HARPER|Albany|  F|   10|
|2021|      ISLA|Albany|  F|    9|
|2021| CHARLOTTE|Albany|  F|    8|
|2021|  ISABELLA|Albany|  F|    8|
|2021|    GIANNA|Albany|  F|    8|
|2021|   ELEANOR|Albany|  F|    8|
|2021|   LILLIAN|Albany|  F|    8|
|2021|  MADELINE|Albany|  F|    8|
|2021|    EVELYN|Albany|  F|    7|
|2021|     RILEY|Albany|  F|    7|
|2021|    HAILEY|Albany|  F|    7|
|2021|    EVERLY|Albany|  F|    7|
+----+----------+------+---+-----+
only showing top 20 rows



In [49]:
# TOP 5 FEMALE NAMES

df_updated_schema.where("sex = 'F'").groupBy("first_name")\
    .agg({'count': 'sum'})\
    .withColumnRenamed("sum(count)", 'total')\
    .sort(desc('total'))\
    .limit(5).show()

+----------+-----+
|first_name|total|
+----------+-----+
|    OLIVIA|16285|
|    SOPHIA|16221|
|  ISABELLA|16185|
|      EMMA|15721|
|       AVA|13009|
+----------+-----+



In [48]:
# TOP 5 MALE NAMES

df_updated_schema.where("sex = 'M'").groupBy("first_name")\
    .agg({'count': 'sum'})\
    .withColumnRenamed("sum(count)", 'total')\
    .sort(desc('total'))\
    .limit(5).show()

+----------+-----+
|first_name|total|
+----------+-----+
|   MICHAEL|17760|
|     JACOB|16846|
|      LIAM|15840|
|    JOSEPH|15829|
|   MATTHEW|15649|
+----------+-----+



In [70]:
# MOST COMMON NAME BY YEAR

df_updated_schema.groupby("year")\
    .agg(max_by("first_name", "count").alias('name'))\
    .show()

+----+------+
|year|  name|
+----+------+
|2007|DANIEL|
|2008|JAYDEN|
|2009| DAVID|
|2010| DAVID|
|2011|JAYDEN|
|2012| DAVID|
|2013| DAVID|
|2014| DAVID|
|2015| ETHAN|
|2016| DAVID|
|2017| DAVID|
|2018| DAVID|
|2019| DAVID|
|2020|  LIAM|
|2021|  LIAM|
+----+------+



In [73]:
# LEAST COMMON NAME BY YEAR

df_updated_schema.groupby("year")\
    .agg(min_by("first_name", "count").alias('name'))\
    .show()

+----+--------+
|year|    name|
+----+--------+
|2007| AALIYAH|
|2008|    ZOEY|
|2009|   MAYER|
|2010|  JOHANN|
|2011| WILLIAM|
|2012|  ROBERT|
|2013|   KATIE|
|2014| JACKSON|
|2015|BENJAMIN|
|2016|  LANDON|
|2017| LINCOLN|
|2018|  HARPER|
|2019|   BYRON|
|2020|   RENZO|
|2021|  OLIVER|
+----+--------+

